In [1]:
from haversine import haversine_vector
import pandas as pd
import numpy as np
import glob
import os
import sys
sys.path.append('../resources/library')
from tropical_cyclone.georeferencing import round_to_grid
from tropical_cyclone.cyclone import init_track_dataframe, tracking_algorithm, paper_tracking_algorithm, track_matching
from tropical_cyclone.visualize import plot_tracks

import warnings
warnings.filterwarnings('ignore')

/Users/davide/anaconda3/envs/mlenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# define inference directory to draw detections
dataset_dir = '../data/inference'
available_models = sorted([folder for folder in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, folder))])
available_models

['02_swin_msl_vo_850',
 '03_vgg_v3_relu_ks3_msl_vo_850',
 '04_vgg_v3_relu_ks5_msl_vo_850',
 '05_vgg_v3_linear_ks3_msl_vo_850',
 '06_swin_msl_vo_850',
 '07_vgg_v3_silu_ks3_msl_vo_850',
 '08_vgg_v1_relu_ks3_msl_vo_850',
 '09_vgg_v1_relu_ks3_msl_vo_850_lnc3',
 '10_swin_lnc3_msl_vo_850',
 '11_swin_fg10_t_500_msl_vo_850']

In [3]:
# select the model to analyze
selected_model = '11_swin_fg10_t_500_msl_vo_850'

# get ibtracs directory
ibtracs_src = '../data/ibtracs/filtered/ibtracs_main-tracks_6h_1980-2021_TS-NR-ET-MX-SS-DS.csv'

# define test years (same as paper)
test_years = [i for i in range(1980,2020)]
# test_years = [1983, 1984, 1993, 1994, 2003, 2004, 2013, 2014]
# test_years = [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
# test_years = [1993]

# kilometer threshold
max_distance_detection = 1000.0

# whether or not to make plots
plot = False

In [4]:
# get model directory
model_dir = os.path.join(dataset_dir, selected_model)
# get inference filenames
inference_files = [os.path.join(model_dir, f'{year}.csv') for year in test_years]
model_dir, inference_files

('../data/inference/11_swin_fg10_t_500_msl_vo_850',
 ['../data/inference/11_swin_fg10_t_500_msl_vo_850/1980.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1981.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1982.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1983.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1984.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1985.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1986.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1987.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1988.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1989.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1990.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1991.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1992.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1993.csv',
  '../data/inference/11_swin_fg10_t_500_msl_vo_850/1994.csv',
  '../data/inferen

In [5]:
# load csv files
csv_files = []
for file in inference_files:
    csv_files.append(pd.read_csv(file, index_col=0))
# merge csv files together
detections = pd.concat(csv_files).reset_index(drop=True)
# convert iso time with pandas
detections['ISO_TIME'] = pd.to_datetime(detections['ISO_TIME'])
# add WS as np.inf
detections['WS'] = np.inf
detections

,ISO_TIME,LAT,LON,WS
0,1980-03-16 00:00:00,49.25,200.50,inf
1,1980-03-16 00:00:00,6.00,147.75,inf
2,1980-03-16 00:00:00,9.75,161.25,inf
3,1980-03-16 06:00:00,49.25,168.25,inf
4,1980-03-16 06:00:00,49.50,206.25,inf
...,...,...,...,...
262702,2019-12-29 00:00:00,4.50,161.25,inf
262703,2019-12-29 06:00:00,43.25,177.00,inf
262704,2019-12-29 06:00:00,15.00,110.75,inf
262705,2019-12-29 06:00:00,12.25,180.25,inf


In [6]:
columns = ['ISO_TIME','SID','NATURE','WMO_WIND','LAT','LON']
# load ibtracs
observations = pd.read_csv(ibtracs_src, index_col=0)
# convert iso time with pandas
observations['ISO_TIME'] = pd.to_datetime(observations['ISO_TIME'])
# get only some columns from ibtracs
observations = observations[columns]
# round lat and lon to be comparable with training data
observations['LAT'] = round_to_grid(observations['LAT'], grid_res=0.25)
observations['LON'] = round_to_grid(observations['LON'], grid_res=0.25)
observations

,ISO_TIME,SID,NATURE,WMO_WIND,LAT,LON
0,1980-03-16 00:00:00,1980076N06148,NR,,6.00,147.75
1,1980-03-16 06:00:00,1980076N06148,NR,,6.00,147.25
2,1980-03-16 12:00:00,1980076N06148,NR,,6.00,146.75
3,1980-03-16 18:00:00,1980076N06148,NR,,6.00,146.50
4,1980-03-17 00:00:00,1980076N06148,NR,,6.00,146.00
...,...,...,...,...,...,...
86188,2021-12-16 06:00:00,2021349N05108,TS,,4.50,105.00
86189,2021-12-16 12:00:00,2021349N05108,TS,,4.25,104.50
86190,2021-12-16 18:00:00,2021349N05108,TS,,4.25,103.00
86191,2021-12-17 00:00:00,2021349N05108,TS,,4.25,103.00


In [7]:
tmp = pd.merge(left=detections, right=observations, on='ISO_TIME', how='inner')
tmp = tmp[tmp['ISO_TIME'].dt.year.isin(test_years)]
dates = tmp['ISO_TIME'].to_numpy()

In [8]:
print(f'There are {len(observations)} observations and {len(detections)} detections')

There are 86193 observations and 262707 detections


In [9]:
# get only detections and observations present on both dataframes
detections = detections[detections['ISO_TIME'].isin(dates)].reset_index(drop=True)
observations = observations[observations['ISO_TIME'].isin(dates)].reset_index(drop=True)

In [10]:
print(f'There are {len(observations)} observations and {len(detections)} detections')

There are 82044 observations and 262675 detections


# Localization

In [11]:
# merge together detections and ibtracs
matches = pd.merge(left=detections, right=observations, on='ISO_TIME')
# compute haversine distance between any couple of points
matches['HDIST'] = haversine_vector(array1=matches[['LAT_x','LON_x']].to_numpy(), array2=matches[['LAT_y','LON_y']].to_numpy(), normalize=True)
matches.head()

,ISO_TIME,LAT_x,LON_x,WS,SID,NATURE,WMO_WIND,LAT_y,LON_y,HDIST
0,1980-03-16 00:00:00,49.25,200.50,inf,1980076N06148,NR,,6.0,147.75,6874.866992
1,1980-03-16 00:00:00,6.00,147.75,inf,1980076N06148,NR,,6.0,147.75,0.000000
2,1980-03-16 00:00:00,9.75,161.25,inf,1980076N06148,NR,,6.0,147.75,1544.003018
3,1980-03-16 06:00:00,49.25,168.25,inf,1980076N06148,NR,,6.0,147.25,5197.762377
4,1980-03-16 06:00:00,49.50,206.25,inf,1980076N06148,NR,,6.0,147.25,7301.127509


In [12]:
# remove all the distances above `max_distance_localization` km
matches = matches[matches['HDIST'] < max_distance_detection]
# group by LATx and LONx and find the minimum (to remove x duplicates)
matches = matches.groupby(by=['ISO_TIME','LAT_x','LON_x','SID','NATURE','WMO_WIND']).min('HDIST').reset_index()
# repeat grouping by LATy and LONy and find the minimum (to remove y duplicates)
matches = matches.groupby(by=['ISO_TIME','LAT_y','LON_y','SID','NATURE','WMO_WIND']).min('HDIST').reset_index()
# show result
matches.head()

,ISO_TIME,LAT_y,LON_y,SID,NATURE,WMO_WIND,LAT_x,LON_x,WS,HDIST
0,1980-03-16 00:00:00,6.0,147.75,1980076N06148,NR,,6.00,147.75,inf,0.000000
1,1980-03-16 06:00:00,6.0,147.25,1980076N06148,NR,,6.25,147.25,inf,27.798770
2,1980-03-16 12:00:00,6.0,146.75,1980076N06148,NR,,6.25,146.00,inf,87.455849
3,1980-03-16 18:00:00,6.0,146.50,1980076N06148,NR,,5.75,147.00,inf,61.898843
4,1980-03-17 00:00:00,6.0,146.00,1980076N06148,NR,,5.00,145.75,inf,114.586206


In [13]:
min_distance_localization = matches['HDIST'].min()
max_distance_localization = matches['HDIST'].max()
mean_distance_localization = matches['HDIST'].mean()
median_distance_localization = matches['HDIST'].median()

print(f"Model {selected_model} Localization results")
print(f"   Min distance ({np.round(min_distance_localization,2)} km)")
print(f"   Max distance ({np.round(max_distance_localization,2)} km)")
print(f"   Average distance ({np.round(mean_distance_localization,2)} km)")
print(f"   Median distance ({np.round(median_distance_localization,2)} km)")

Model 11_swin_fg10_t_500_msl_vo_850 Localization results
   Min distance (0.0 km)
   Max distance (999.36 km)
   Average distance (73.82 km)
   Median distance (39.17 km)


In [14]:
if plot: plot_detections(detections, observations)

# Classification

In [15]:
def F_beta(beta, precision, recall):
    return (1 + beta**2) * ((precision * recall) / ((beta**2 * precision) + recall))

In [16]:
n_dets = len(detections)
n_tp = len(matches)
n_obs = len(observations)
n_fp = n_dets - n_tp
n_fn = n_obs - n_tp

In [17]:
precision = n_tp / (n_tp + n_fp)
recall = n_tp / (n_tp + n_fn)
f2_score = F_beta(beta=2, precision=precision, recall=recall) * 100

In [18]:
print(f"Model {selected_model} Classification results")
print(f"   F2 : {np.round(f2_score,2)} % (precision={np.round(precision, 2)}, recall={np.round(recall,2)})")
print(f"   TP : {n_tp} out of {n_obs} observations ({np.round(n_tp / n_obs * 100)} %)")
print(f"   FP : {n_fp} out of {n_dets} ML detections ({np.round(n_fp / n_dets * 100)} %)")
print(f"   FN : {n_fn} out of {n_obs} observations ({np.round(n_fn / n_obs * 100)} %)")

Model 11_swin_fg10_t_500_msl_vo_850 Classification results
   F2 : 64.74 % (precision=0.29, recall=0.93)
   TP : 76506 out of 82044 observations (93.0 %)
   FP : 186169 out of 262675 ML detections (71.0 %)
   FN : 5538 out of 82044 observations (7.0 %)


# Tracking

In [19]:
# minimum track length (1 day)
min_track_count = 12
# minimum speed of wind in order to consider the track true
min_wind_speed = 17.0
# maximum distance between tracks
max_track_distance_tracking = 400.0

grid_res = 0.25
km_to_deg = 110.474

In [20]:
# rename SID to TRACK_ID
observed_tracks = observations.rename(columns={'SID':'TRACK_ID'})
# get only long enough tracks for the comparison
valid_observations_sids = observed_tracks.groupby('TRACK_ID').filter(lambda x: len(x) >= min_track_count)['TRACK_ID'].unique()
# filter out the observations
observed_tracks = observed_tracks[observed_tracks['TRACK_ID'].isin(valid_observations_sids)].reset_index(drop=True)
observed_tracks.head()

,ISO_TIME,TRACK_ID,NATURE,WMO_WIND,LAT,LON
0,1980-03-16 00:00:00,1980076N06148,NR,,6.0,147.75
1,1980-03-16 06:00:00,1980076N06148,NR,,6.0,147.25
2,1980-03-16 12:00:00,1980076N06148,NR,,6.0,146.75
3,1980-03-16 18:00:00,1980076N06148,NR,,6.0,146.50
4,1980-03-17 00:00:00,1980076N06148,NR,,6.0,146.00


In [21]:
# apply tracking scheme
tracking_src = f'/Users/davide/Developer/ml-tropical-cyclones-detection/data/inference/{selected_model}/tracking.csv'
if not os.path.exists(tracking_src):
    detected_tracks = init_track_dataframe(detections)
    detected_tracks = tracking_algorithm(detected_tracks, max_track_distance_tracking, min_track_count)
    # detected_tracks = paper_tracking_algorithm(detected_tracks, max_track_distance_tracking, min_track_count)
    detected_tracks.to_csv(tracking_src)
else:
    detected_tracks = pd.read_csv(tracking_src, index_col=0)
detected_tracks.head()

# detected_tracks = init_track_dataframe(detections)
# detected_tracks = tracking_algorithm(detected_tracks, max_track_distance_tracking, min_track_count)
# detected_tracks = paper_tracking_algorithm(detected_tracks, max_track_distance_tracking, min_track_count)

,ISO_TIME,LAT,LON,WS,TRACK_ID,HAVERSINE
0,1980-03-16 00:00:00,6.00,147.75,inf,19800316T00_1,61.875826
1,1980-03-16 06:00:00,6.25,147.25,inf,19800316T00_1,138.167604
2,1980-03-16 12:00:00,6.25,146.00,inf,19800316T00_1,123.775737
3,1980-03-16 18:00:00,5.75,147.00,inf,19800316T00_1,161.568695
4,1980-03-17 00:00:00,5.00,145.75,inf,19800316T00_1,111.195096


In [22]:
print(f'There are:')
print(f'   - {len(detected_tracks["TRACK_ID"].unique())} detected tracks')
# print(f'   - {len(paper_detected_tracks["TRACK_ID"].unique())} detected tracks (paper)')
print(f'   - {len(observed_tracks["TRACK_ID"].unique())} observed tracks')

There are:
   - 2918 detected tracks
   - 2334 observed tracks


In [23]:
if plot:plot_tracks(detected_tracks, observed_tracks)
# plot_tracks(detected_tracks[pd.to_datetime(detected_tracks['ISO_TIME']).dt.year.isin([1993])], observed_tracks[pd.to_datetime(observed_tracks['ISO_TIME']).dt.year.isin([1993])])

# Track Matching

In [24]:
sys.path.append('/Users/davide/Developer/ml-tropical-cyclones-detection/resources/library/dynamicopy-0.6.1')
import dynamicopy

# maximum distance to consider true the match
max_track_distance_matching = 300.0

Failure in importing the cartopy library, the dynamicopy.cartoplot will not be loaded.     Please install cartopy if you wish to use it.
Failure in importing the cartopy library, the dynamicopy.cartoplot will not be loaded.     Please install cartopy if you wish to use it.


##### Our track matching algorithm provides same results of Bourdin's (but more slowly)

In [25]:
# track_matches = track_matching(detected_tracks, observed_tracks, max_track_distance_matching)
# # H = HITS = True Positive
# H = len(track_matches[(track_matches['DET_TRACK_ID']!='') & (track_matches['OBS_TRACK_ID']!='')])
# # M = Miss = False Negative
# M = len(track_matches[(track_matches['DET_TRACK_ID']=='') & (track_matches['OBS_TRACK_ID']!='')])
# # FA = False Alarm = False Positive
# FA = len(track_matches[(track_matches['DET_TRACK_ID']!='') & (track_matches['OBS_TRACK_ID']=='')])

# POD = (H / (H + M))
# FAR = (FA / (H + FA))

# print(f"Hits : {H}")
# print(f"Miss : {M}")
# print(f"False Alarm : {FA}")
# print(f"POD : {POD}")
# print(f"FAR : {FAR}")

In [26]:
# load bourdin observed tracks from library
# bourdin_observed_tracks = dynamicopy.load_ibtracs()
# bourdin_observed_tracks = bourdin_observed_tracks[bourdin_observed_tracks['basin'].isin(['WNP','ENP','NATL'])].reset_index(drop=True)
bourdin_observed_tracks = pd.read_csv('/Users/davide/Developer/ml-tropical-cyclones-detection/resources/library/zenodo_bourdin/ibtracs/ibtracs.since1980.cleaned.csv', index_col=0)

# convert columns to dynamicopy compliant format
detected_tracks = detected_tracks.rename(columns={'ISO_TIME':'time','LAT':'lat','LON':'lon','TRACK_ID':'track_id'})
observed_tracks = observed_tracks.rename(columns={'ISO_TIME':'time','LAT':'lat','LON':'lon','TRACK_ID':'track_id'})

# convert time column pandas datetime format
detected_tracks['time'] = pd.to_datetime(detected_tracks['time'])
observed_tracks['time'] = pd.to_datetime(observed_tracks['time'])
bourdin_observed_tracks['time'] = pd.to_datetime(bourdin_observed_tracks['time'])

# convert longitudes to range [0, 360] format
detected_tracks['lon'] = (detected_tracks['lon'] + 540) % 360 - 180
observed_tracks['lon'] = (observed_tracks['lon'] + 540) % 360 - 180
bourdin_observed_tracks['lon'] = (bourdin_observed_tracks['lon'] + 540) % 360 - 180

# remove out of bound (both space and time) detections from bourdin observations
bourdin_observed_tracks = bourdin_observed_tracks[(bourdin_observed_tracks['lon']>=100) & (bourdin_observed_tracks['lon']<=320) & (bourdin_observed_tracks['lat']>=0) & (bourdin_observed_tracks['lat']<=70)]
bourdin_observed_tracks = bourdin_observed_tracks[bourdin_observed_tracks['time'].isin(dates)]
bourdin_observed_tracks = bourdin_observed_tracks[bourdin_observed_tracks['track_id'].isin(bourdin_observed_tracks.groupby('track_id').filter(lambda x: len(x) >= min_track_count)['track_id'].unique())].reset_index(drop=True)

In [27]:
bourdin_track_matches = dynamicopy.match_tracks(detected_tracks, bourdin_observed_tracks, "ours", 'bourdin', max_dist=max_track_distance_matching, min_overlap=0, ref=True)

n_ib_match = len(bourdin_track_matches[f'id_bourdin'].unique())
n_our_match = len(bourdin_track_matches['id_ours'].unique())
n_observations = len(bourdin_observed_tracks.track_id.unique())
n_detections = len(detected_tracks.track_id.unique())

H, M, FA = n_ib_match, n_observations - n_ib_match, n_detections - n_our_match
POD = H / (H + M)
FAR = FA / (H + FA)

print(f"Hits : {H}")
print(f"Misses : {M}")
print(f"False Alarms : {FA}")
print(f"POD : {POD}")
print(f"FAR : {FAR}")

Hits : 965
Misses : 107
False Alarms : 1640
POD : 0.9001865671641791
FAR : 0.6295585412667947


In [28]:
track_matches = dynamicopy.match_tracks(detected_tracks, observed_tracks, "ours", 'ibtracs', max_dist=max_track_distance_matching, min_overlap=0, ref=True)

n_ib_match = len(track_matches[f'id_ibtracs'].unique())
n_our_match = len(track_matches['id_ours'].unique())
n_observations = len(observed_tracks.track_id.unique())
n_detections = len(detected_tracks.track_id.unique())

H, M, FA = n_ib_match, n_observations - n_ib_match, n_detections - n_our_match
POD = H / (H + M)
FAR = FA / (H + FA)

# POD, FAR, H, M, FA
print(f"Hits : {H}")
print(f"Misses : {M}")
print(f"False Alarms : {FA}")
print(f"POD : {POD}")
print(f"FAR : {FAR}")

Hits : 2007
Misses : 327
False Alarms : 392
POD : 0.8598971722365039
FAR : 0.16340141725719048


# Save to file

In [29]:
columns = [
    'model', 'max_distance_detection', 'n_dets', 'n_tp', 'n_obs', 'n_fp', 'n_fn', 'precision', 
    'recall', 'f2_score', 'min_distance_localization', 'max_distance_localization', 
    'mean_distance_localization', 'median_distance_localization', 'min_track_count', 
    'max_distance_tracking', 'min_wind_speed', 'max_track_distance_matching', 
    'max_track_distance_tracking', 'H', 'M', 'FA', 'POD', 'FAR', 'ibtracs_src', 'test_years', 
]
dst = '/Users/davide/Developer/ml-tropical-cyclones-detection/data/inference/results_analysis.csv'

if os.path.exists(dst):
    results = pd.read_csv(dst, index_col=0)
else:
    results = pd.DataFrame(columns=columns)

results = pd.concat([results, pd.DataFrame(data={
    'model': [selected_model], 'max_distance_detection': [max_distance_detection], 'n_dets': [n_dets], 'n_tp': [n_tp], 
    'n_obs': [n_obs], 'n_fp': [n_fp], 'n_fn': [n_fn], 'precision': [precision], 'recall': [recall], 
    'f2_score': [f2_score], 'min_distance_localization': [min_distance_localization], 'max_distance_localization': [max_distance_localization], 
    'mean_distance_localization': [mean_distance_localization], 'median_distance_localization': [median_distance_localization], 
    'min_track_count': [min_track_count], 'max_distance_tracking': [max_track_distance_tracking], 'min_wind_speed': [min_wind_speed], 
    'max_track_distance_matching': [max_track_distance_matching], 'max_track_distance_tracking': [max_track_distance_tracking], 
    'H': [H], 'M': [M], 'FA': [FA], 'POD': [POD], 'FAR': [FAR], 'ibtracs_src': [ibtracs_src], 'test_years': [test_years], 
})])

results = results.reset_index(drop=True)
# results.to_csv(dst)

# Paper Results

In the paper, with the ML ensemble we have the following results:

- F2-score : 53 %
- Euclidean distance : 117.06 km
- Hit rate : 88.91 %
- POD : 71.49 %
- FAR : 23.00 %